# Face Recognition

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
import face_recognition
import cv2
import time
print("imported")

Using TensorFlow backend.


imported


# Get Face-encodings for all faces and store it in a CSV file

Get Encodings for all images and store in a CSV file. With name and Encodings
Note: I've got only one image per a  person

# Algorithm


1. Loop over folder's contents
2. For each file , get encodings
3. In a CSV file, save file name, encodings



In [2]:
#create an empty dataframe 
data = {'Name':  [],
        'Encodings' : []
        }

df = pd.DataFrame (data, columns = ['Name','Encodings'])
print(df)

Empty DataFrame
Columns: [Name, Encodings]
Index: []


In [3]:
Encodings_CSV_path = ""
folder_path = "C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face Recognition\\SingleShotDataset\\*.jpg"

In [4]:
files = glob.glob(folder_path)
list_dicts = []
name_list = []
dfObj = pd.DataFrame(columns = ['Enc'])
i = 0
for file in files:
    image = face_recognition.load_image_file(file)
    encodings = face_recognition.face_encodings(image)[0]
    encodings = encodings.tolist()
    print(type(encodings))
    list_dicts.append(encodings)
    basename = os.path.basename(file)
    name = basename[:-4]
    print(name)
    df2 = (name,encodings)
    name_list.append(name) 
    dfObj1 = pd.Series(encodings)
    dfObj.append(dfObj1,ignore_index=True)
    print (i)
    #list_dicts.append(df2)
    dataffame = pd.DataFrame(list_dicts)
    print(dataffame)
    i+=1
print(dataffame.shape)
dataffame['name_list'] = pd.Series(name_list,index=dataffame.index)
print(dataffame.shape)

dataffame.set_index('name_list',inplace = True)
print(dataffame.shape)

# SAve it to a CSV file

dataffame.to_csv("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face Recognition\\Encodings\\data.csv")

<class 'list'>
Dakshayani
0
        0         1         2         3         4        5         6    \
0 -0.094346  0.030154  0.104514 -0.017434 -0.049997 -0.05203  0.038582   

        7         8         9    ...       118       119       120       121  \
0 -0.145181  0.224201 -0.140293  ... -0.018829  0.047649 -0.065139 -0.070123   

        122       123       124       125       126       127  
0 -0.090295 -0.055078  0.048481 -0.009668  0.046949 -0.015656  

[1 rows x 128 columns]
<class 'list'>
Ramya
1
        0         1         2         3         4         5         6    \
0 -0.094346  0.030154  0.104514 -0.017434 -0.049997 -0.052030  0.038582   
1 -0.031513  0.046748  0.083386 -0.064198 -0.081684  0.006021  0.008460   

        7         8         9    ...       118       119       120       121  \
0 -0.145181  0.224201 -0.140293  ... -0.018829  0.047649 -0.065139 -0.070123   
1 -0.076569  0.215477 -0.194164  ...  0.028641  0.077017 -0.007868 -0.101604   

        122       12

# Code to recognise face in real time video

Now, we're gonna read the csv file and store in the encodings to compare faces from each frame

The FaceRecognition api has faceencodings of size 128 and hence for a person 1*128 size array is needed



In [5]:
# read csv file and load into a dataframe
df = pd.read_csv("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face Recognition\\Encodings\\data.csv")
print(df.head())

    name_list         0         1         2         3         4         5  \
0  Dakshayani -0.094346  0.030154  0.104514 -0.017434 -0.049997 -0.052030   
1       Ramya -0.031513  0.046748  0.083386 -0.064198 -0.081684  0.006021   

          6         7         8  ...       118       119       120       121  \
0  0.038582 -0.145181  0.224201  ... -0.018829  0.047649 -0.065139 -0.070123   
1  0.008460 -0.076569  0.215477  ...  0.028641  0.077017 -0.007868 -0.101604   

        122       123       124       125       126       127  
0 -0.090295 -0.055078  0.048481 -0.009668  0.046949 -0.015656  
1 -0.144912 -0.080937  0.109795 -0.026641  0.096905 -0.020574  

[2 rows x 129 columns]


In [6]:
list_encodings = []
for i in range(len(df)):
    list_encodings.append(df.iloc[i,1:].values.astype('float64'))


# Running Inference

In [8]:

video_capture = cv2.VideoCapture(0)


known_face_names = df['name_list']

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(list_encodings, face_encoding)
            name = "Unknown"# unknown if not in the database

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(list_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()